<a href="https://colab.research.google.com/github/quiquefluque/Primera-prueba-/blob/main/C%C3%B3digo_escalble_de_pruebas_de_bots_en_Monte_Carlo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
### Codigó escalable para probar bots en simulaciones de monte carlo. Permite variar el numero de simulaciones y la media en la que lo vamos a probar###
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import yfinance as yf

df = yf.download('SPY', period='2y')
df = df[['Close']].copy()
df.columns = ['Precio']

#1. EL ADN (Tu turno): Calcula los retornos, la media y la volatilidad
df["retornos"] = df["Precio"].pct_change()
media_precios=df["Precio"].mean()
media_retornos=df["retornos"].mean()
volatilidad =np.std(df["retornos"]) #es solo un numero, no es el error de cada volatilidad a la volatilidad media

print(f"La media de precio de los 2 ultimos años es {media_precios}, y la volatilidad es {volatilidad}")

# 2. MONTE CARLO (Tu turno): Crea 100 escenarios futuros de 252 días
# Esta vez no los dibujes todos, solo genéralos.
precio_actual = df['Precio'].iloc[-1]
simulaciones_dict = {}#creo tabla vacia
for i in range(1000):
  retonos_simulados =np.random.normal(media_retornos,volatilidad, size=252)
  precios_simulados = precio_actual * (1 + retornos_simulados).cumprod(axis=0)
  simulaciones_dict[f"Sim_{i}"] = precios_simulados #añado columna por cada simulacion con los precios

#3. BOTEAR TABLA:
media_ventana = 60
resultados_finales = []#listas vacias a las que voy añadiendo con el append
drawdowns_maximos = []#listas vacias a las que voy añadiendo con el append

# Recorremos cada una de las 100 simulaciones
for col in df_simulaciones.columns:
    # 1. Calculamos la media para ESTE mundo
    media_fut = df_simulaciones[col].rolling(window=media_ventana).mean()

    # 2. Generamos órdenes (1 compra, 0 fuera)
    orden = np.where(df_simulaciones[col] > media_fut, 1, 0)

    # 3. Calculamos retornos y beneficios (Wins)
    ret_sim = df_simulaciones[col].pct_change()
    # Usamos shift(1) porque compramos al día siguiente de la señal
    ret_robot = (ret_sim * pd.Series(orden).shift(1)).fillna(0)
    wins = (1 + ret_robot).cumprod()

    # 4. Guardamos el resultado final y el susto (Drawdown) (append es para añadir a una lista)
    resultados_finales.append(wins.iloc[-1])

    pico = wins.cummax()
    dd = (pico - wins) / pico
    drawdowns_maximos.append(dd.max())

# --- ESTADÍSTICAS PROFESIONALES ---
resultados_finales = np.array(resultados_finales)
prob_exito = (resultados_finales > 1).mean() * 100
ganancia_media = (resultados_finales.mean() - 1) * 100
peor_susto_medio = np.array(drawdowns_maximos).mean() * 100

print(f"\n--- RESULTADOS PARA MEDIA DE {media_ventana} DÍAS ---")
print(f"Probabilidad de salir ganando: {prob_exito:.2f}%")
print(f"Ganancia media esperada: {ganancia_media:.2f}%")
print(f"Susto (Drawdown) medio: {peor_susto_medio:.2f}%")

df_simulaciones = pd.DataFrame(simulaciones_dict)
print(df_simulaciones)









/tmp/ipython-input-1985410211.py:6: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download('SPY', period='2y')
[*********************100%***********************]  1 of 1 completed


La media de precio de los 2 ultimos años es 587.3469523964766, y la volatilidad es 0.010324660236780425

--- RESULTADOS PARA MEDIA DE 60 DÍAS ---
Probabilidad de salir ganando: 0.00%
Ganancia media esperada: -15.47%
Susto (Drawdown) medio: 18.67%
          Sim_0       Sim_1       Sim_2       Sim_3       Sim_4       Sim_5  \
0    686.976730  686.976730  686.976730  686.976730  686.976730  686.976730   
1    678.051191  678.051191  678.051191  678.051191  678.051191  678.051191   
2    672.746966  672.746966  672.746966  672.746966  672.746966  672.746966   
3    674.676883  674.676883  674.676883  674.676883  674.676883  674.676883   
4    675.733996  675.733996  675.733996  675.733996  675.733996  675.733996   
..          ...         ...         ...         ...         ...         ...   
247  654.189072  654.189072  654.189072  654.189072  654.189072  654.189072   
248  665.303246  665.303246  665.303246  665.303246  665.303246  665.303246   
249  659.523444  659.523444  659.523444  6